In [3]:
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
import numpy as np
import matplotlib.pyplot as plt

# 데이터셋 생성
X, y = make_moons(n_samples=10000, noise=0.4, random_state=42)
# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 결정 트리 하이퍼파라미터 최적화
param_grid = {
    'max_depth': [6, 9, None],
    'max_leaf_nodes': [10, 20, None],
    'min_samples_split': [2, 5]
}

grid_search = GridSearchCV(DecisionTreeClassifier(random_state=42), param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# 최적의 하이퍼파라미터 출력
print("Best Parameters:", grid_search.best_params_)

# 최적 모델로 테스트 정확도 확인
best_tree = grid_search.best_estimator_
print("Single Tree Accuracy:", best_tree.score(X_test, y_test))

# 랜덤 포레스트 구현
np.random.seed(42)
subsets = [np.random.choice(len(X_train), len(X_train), replace=True) for _ in range(100)]

# 서브셋별 결정 트리 학습
single_tree_accuracies = []
for subset in subsets:
    tree = DecisionTreeClassifier(**grid_search.best_params_, random_state=42)
    tree.fit(X_train[subset], y_train[subset])
    single_tree_accuracies.append(tree.score(X_test, y_test))

print("Average Single Tree Accuracy:", np.mean(single_tree_accuracies))

# 다수결 앙상블
from scipy.stats import mode

all_predictions = []
for subset in subsets:
    tree = DecisionTreeClassifier(**grid_search.best_params_, random_state=42)
    tree.fit(X_train[subset], y_train[subset])
    all_predictions.append(tree.predict(X_test))

# 다수결 방식으로 최종 예측 생성
final_predictions = np.squeeze(mode(all_predictions, axis=0).mode)

# 앙상블 정확도 계산
from sklearn.metrics import accuracy_score
ensemble_accuracy = accuracy_score(y_test, final_predictions)
print("Ensemble Accuracy:", ensemble_accuracy)

Best Parameters: {'max_depth': 9, 'max_leaf_nodes': 20, 'min_samples_split': 2}
Single Tree Accuracy: 0.87
Average Single Tree Accuracy: 0.8637999999999999
Ensemble Accuracy: 0.8695
